# Isolate moving objects in video
Short code for removing moving objects in video (or removing static objects in video).

We first load some Julia packages and specify the video file path.

In [ ]:
using Plots, Images, VideoIO, LinearAlgebra, LowRankApprox;

In [ ]:
path = ""
file = path * "";

Load the video file and define height, width, fps and number of frames.

In [ ]:
f = VideoIO.openvideo(file)
h = f.height
w = f.width
fps = f.stream_info.stream.avg_frame_rate.num
n = f.stream_info.stream.nb_frames-1
RGBn = 3;

Define a function to read the video file, frame by frame, and a function where the SVD is performed, and finally run them.

In [ ]:
imframe(f) = Float32.(channelview(read(f)))

function todata(f)
    imdata = Array{Float32}(undef, RGBn, h, w, n)
    for k in 1:n # dont need while !eof(f), because we know n
        imdata[:, :, :, k] = imframe(f)
    end
    
    imdata
end

function lowrank(vid, n)
    out = []
    for i in 1:RGBn
        v = reshape(vid[i, :, :, :], :, n)
        u, s, vt = psvd(v, rtol=1e-6) # randomised alg
        #u, s, vt = svd(v); # standard alg
        
        push!(out, (u=u, s=s, vt=vt))
    end
    
    out
end

vid = todata(f)#[:, :, :, 1:5:end]
#n = size(vid, 4)
svddata = lowrank(vid, n);

Recreate the video in the low-dimensional space, removing all non-static components, such as moving objects and scale pixels between 0 and 1.

In [ ]:
function filter()
    vb = Array{N0f8}(undef, RGBn, h, w, n)
    vf = Array{N0f8}(undef, RGBn, h, w, n)
    
    ns = 1 # number of singular values
    for i = 1:RGBn
        vb_ = reshape(svddata[i].u[:, 1:ns] * diagm(svddata[i].s[1:ns]) * svddata[i].vt[:, 1:ns]', h, w, n)

        # get only static component
        vf_ = vid[i, :, :, :] - vb_; 

        # scale to [0, 1]
        vb[i, :, :, :] = N0f8.(scaleminmax(0, 1).(vb_))
        vf[i, :, :, :] = N0f8.(scaleminmax(0, 1).(vf_));
    end
    
    vb, vf, N0f8.(vid)
end

vb, vf, vo = filter();

Prepare video to be exported.

In [ ]:
imgstacko = [colorview(RGB, vo[:, :, :, i]) for i in 1:n]
imgstackb = [colorview(RGB, vb[:, :, :, i]) for i in 1:n]
imgstackf = [colorview(RGB, vf[:, :, :, i]) for i in 1:n];

Export the three video files. As of 2020-08-22 something is wrong with the export, see https://github.com/JuliaIO/VideoIO.jl/pull/179. Using handbrake on the lossless file works.

In [ ]:
props = [:priv_data => ("crf"=>"0","preset"=>"medium")]
encodevideo(path * "_o.mp4",
    imgstacko, framerate=fps, AVCodecContextProperties=props, codec_name = "libx264rgb");
encodevideo(path * "_b.mp4",
    imgstackb, framerate=fps, AVCodecContextProperties=props, codec_name = "libx264rgb");
encodevideo(path * "_f.mp4",
    imgstackf, framerate=fps, AVCodecContextProperties=props, codec_name = "libx264rgb");